# **LLM that can perform empathetic medical question‑answering**

Author: Tianze Chen  
Github: https://github.com/Tianze-Chen819

## Dataset 
1.Collect training data  
Each row should contain a medical question (from a patient) and an empathic answer (from a clinician)  
Question,Answer  
"I feel constant headaches…", "I’m sorry you’re experiencing that..."

## Model & Tokenizer

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(base_model_name, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    load_in_4bit=True,  # quantization to save memory
    device_map="auto"
)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2.
401 Client Error. (Request ID: Root=1-6898ecf0-70fc95c86615136c4d889d6b;ccc17605-e8a9-4e4f-ac8e-5eb221f9fdc5)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.2 is restricted. You must have access to it and be authenticated to access it. Please log in.

## Format Dialogues

In [5]:
SYSTEM_PROMPT = "You are a compassionate medical assistant."

def format_pair(question, answer):
    return f"<s>[INST] <<SYS>>\n{SYSTEM_PROMPT}\n<</SYS>>\n{question} [/INST] {answer}</s>"

## LoRA Fine‑Tuning

In [ ]:
from peft import LoraConfig, get_peft_model
from transformers import Trainer, TrainingArguments

# LoRA configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none"
)
model = get_peft_model(model, lora_config)

# Training hyperparameters
args = TrainingArguments(
    output_dir="empathetic-medical-bot",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch"
)

# Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()
model.save_pretrained("empathetic-medical-bot")
tokenizer.save_pretrained("empathetic-medical-bot")


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4.3 MB/s eta 0:00:00a 0:00:010m

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


NameError: name 'model' is not defined

## Inference

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("empathetic-medical-bot")
tokenizer = AutoTokenizer.from_pretrained("empathetic-medical-bot")

prompt = "I've been feeling anxious about my heart health lately."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
output = model.generate(**inputs, max_new_tokens=128)
print(tokenizer.decode(output[0], skip_special_tokens=True))


OSError: empathetic-medical-bot is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`